In [1]:
import ECGDataset 
import Models 
import Net
from train_test_validat import *
from self_attention import *
import matplotlib.pyplot as plt
import ecg_plot
import cam
import ECGplot
import ECGHandle
import torch
import torch.utils.data as Data
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold
import random
import pandas as pd
from tqdm import tqdm

import time
import math
import os
import gc
from torch.utils.tensorboard import SummaryWriter


def seed_torch(seed=2023):
	random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed) # 为了禁止hash随机化，使得实验可复现
	np.random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
	torch.backends.cudnn.benchmark = False 
	torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.enabled = False

In [2]:
EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)
DEVICE = "cpu"
seed_torch(2023)

cuda:0


In [3]:
data_root = '/workspace/data/Preprocess_HTN/datas_/'
ALL_data = pd.read_csv(data_root+'/All_data_handled_ID_range_age_IDimputate.csv',low_memory=False)
ALL_data = ECGHandle.change_label(ALL_data)
ALL_data = ECGHandle.filter_ID(ALL_data)
ALL_data = ECGHandle.filter_QC(ALL_data)
ALL_data = ECGHandle.filter_ages(ALL_data,18)
ALL_data = ECGHandle.filter_departmentORlabel(ALL_data,'外科')
ALL_data = ECGHandle.correct_label(ALL_data)
ALL_data = ECGHandle.correct_age(ALL_data)
ALL_data = ECGHandle.filter_diagnose(ALL_data,'起搏')
ALL_data = ECGHandle.filter_diagnose(ALL_data,'房颤')
# ALL_data = ECGHandle.filter_diagnose(ALL_data,'阻滞')
ALL_data = ECGHandle.remove_duplicated(ALL_data)
ALL_data = ALL_data.rename(columns={'住院号':'ID','年龄':'age','性别':'gender','姓名':'name'}) 
ALL_data_buffer = ALL_data.copy()
seed_torch(2023)
ALL_data_buffer = ALL_data_buffer.sample(frac=1).reset_index(drop=True) #打乱顺序
# all_dataset = ECGHandle.ECG_Dataset(data_root,ALL_data_buffer,preprocess = True)
####################################################################随机选取test
test_df,tv_df = Pair_ID(ALL_data,0.2,Range_max=15,pair_num=1)
test_dataset = ECGHandle.ECG_Dataset(data_root,test_df,preprocess = True)



            orginal   removed diagnose NaN
   nums      200082          199997       
              HTN             NHTN        
   nums       3273           196724       


            orginal      removed ID NaN   
   nums      199997          199995       
              HTN             NHTN        
   nums       3273           196722       


            orginal            QC         
   nums      199995          72845        
              HTN             NHTN        
   nums       1497           71348        


            orginal      filtered ages    
   nums      72845           69819        
              HTN             NHTN        
   nums       1477           68342        


            orginal   filtered department 
   nums      69819           15344        
              HTN             NHTN        
   nums       1477           13867        


     reset num:       10  
  ERR labels num:     27  
            orginal      correct label    
   nums      15344           15

In [4]:
torch.cuda.empty_cache()# 清空显卡cuda
NET = [Net.MLBFNet_GUR_o(True,True,True,2,Dropout_rate=0.3),] # type: ignore
testmodel = NET[0].to(DEVICE)

In [5]:
Models_path = '/workspace/data/Interpretable_HTN/model/20230214_150848/20230214_150848/BestF1_0.pt'

In [6]:
testmodel.load_state_dict(torch.load(Models_path))
test_dataloader = Data.DataLoader(dataset=test_dataset, shuffle= False,batch_size=128)
test_acc = []   
criterion = torch.nn.CrossEntropyLoss() 
y_true,y_pred,t_out,test_loss,test_acc = eval_model(test_dataloader,criterion,testmodel,DEVICE) # 测试模型
print('loss =',test_loss,'acc =',test_acc)
print('f1_macro =',f1_score(y_true, y_pred, average='macro')) 
print('f1_micro =',f1_score(y_true, y_pred, average='micro')) 
print('f1_binary =',f1_score(y_true, y_pred, average='binary')) 
cm = confusion_matrix(y_true=y_true, y_pred=y_pred)
print("Confusion Matrix: ")
print(cm)

loss = 0.4674440324306488 acc = 0.8477711397058824
f1_macro = 0.8449840280635742
f1_micro = 0.8451327433628318
f1_binary = 0.8401826484018264
Confusion Matrix: 
[[198  28]
 [ 42 184]]


In [7]:
auc = roc_auc_score(y_true,y_score=((np.array(t_out))[:,1]))
print(auc)

0.9115435821129297


In [ ]:
testmodel.load_state_dict(torch.load(Models_path))
ALL_dataloader = Data.DataLoader(dataset=all_dataset, shuffle= False,batch_size=128)
test_acc = []   
criterion = torch.nn.CrossEntropyLoss() 
y_true,y_pred,y_out,test_loss,test_acc = eval_model(ALL_dataloader,criterion,testmodel,DEVICE) # 测试模型
print('loss =',test_loss,'acc =',test_acc)
print('f1_macro =',f1_score(y_true, y_pred, average='macro')) 
print('f1_micro =',f1_score(y_true, y_pred, average='micro')) 
print('f1_binary =',f1_score(y_true, y_pred, average='binary')) 
cm = confusion_matrix(y_true=y_true, y_pred=y_pred)
print("Confusion Matrix: ")
print(cm)

GET CAM Value

In [ ]:
testmodel.eval()
# 定义获取梯度的函数
fmap_block = list()
grad_block = list()

# 获取反向的传播图
def backward_hook(module, grad_in, grad_out):
    grad_block.append(grad_out[0].detach())

# 定义获取特征图的函数
def farward_hook(module, input, output):
    fmap_block.append(output)

In [ ]:
testmodel.load_state_dict(torch.load(Models_path))

In [ ]:
testmodel.layers0.register_forward_hook(farward_hook)	#正向传播
testmodel.layers1.register_forward_hook(farward_hook) 
testmodel.layers2.register_forward_hook(farward_hook)	#正向传播
testmodel.layers3.register_forward_hook(farward_hook) 
testmodel.layers4.register_forward_hook(farward_hook)	#正向传播
testmodel.layers5.register_forward_hook(farward_hook) 
testmodel.layers6.register_forward_hook(farward_hook)	#正向传播
testmodel.layers7.register_forward_hook(farward_hook) 
testmodel.layers8.register_forward_hook(farward_hook)	#正向传播
testmodel.layers9.register_forward_hook(farward_hook) 
testmodel.layers10.register_forward_hook(farward_hook)	#正向传播
testmodel.layers11.register_forward_hook(farward_hook) 
testmodel.conv3.register_forward_hook(farward_hook) 
testmodel.layers_list_2d[0].register_forward_hook(farward_hook) 
testmodel.layers_list_2d[1].register_forward_hook(farward_hook) 
testmodel.layers_list_2d[2].register_forward_hook(farward_hook)  

In [ ]:
testmodel.layers0.register_full_backward_hook(backward_hook)#反向传播
testmodel.layers1.register_full_backward_hook(backward_hook)#反向传播
testmodel.layers2.register_full_backward_hook(backward_hook)#反向传播
testmodel.layers3.register_full_backward_hook(backward_hook)#反向传播 
testmodel.layers4.register_full_backward_hook(backward_hook)#反向传播
testmodel.layers5.register_full_backward_hook(backward_hook)#反向传播
testmodel.layers6.register_full_backward_hook(backward_hook)#反向传播
testmodel.layers7.register_full_backward_hook(backward_hook)#反向传播 
testmodel.layers8.register_full_backward_hook(backward_hook)#反向传播
testmodel.layers9.register_full_backward_hook(backward_hook)#反向传播
testmodel.layers10.register_full_backward_hook(backward_hook)#反向传播
testmodel.layers11.register_full_backward_hook(backward_hook)#反向传播
testmodel.conv3.register_full_backward_hook(backward_hook)#反向传播
testmodel.layers_list_2d[0].register_full_backward_hook(backward_hook)#反向传播
testmodel.layers_list_2d[1].register_full_backward_hook(backward_hook)#反向传播
testmodel.layers_list_2d[2].register_full_backward_hook(backward_hook)#反向传播

In [ ]:
branch_fmap_sum_HTN = np.zeros(27)
branch_fmap_sum_NHTN = np.zeros(27)

for itme in (range(test_dataset.__len__())):
    layer2d_vlue_list = list()
    # testmodel.layer4.register_forward_hook(farward_hook)	#正向传播
    # testmodel.layer4.register_full_backward_hook(backward_hook)#反向传播
    fmap_block = list()
    grad_block = list()
    inputs,labels = test_dataset.__getitem__(itme)
    labels = torch.tensor(labels)
    labels = labels.unsqueeze(0) # 在首位添加1维作为batchsize
    inputs = inputs.unsqueeze(0) # 在首位添加1维作为batchsize

    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)  
    testmodel.eval()
    outputs = testmodel(inputs)
    possibility = nn.functional.softmax(outputs,dim=-1)
    _,pred = outputs.max(1)     # 求概率最大值对应的标签
    loss = outputs[0,pred]      # 网络对应于pred的类别的输出即为loss
    loss.backward(retain_graph=True)  #retain_graph=True，目的为是为保留该过程中计算的梯度，后续G网络更新时使用
    for i in range(12):
        layer2d_vlue = cam.caculate_layer_cam_vlue(fmap_block[i][0].clone().detach().cpu().numpy(),grad_block[15-i][0].clone().detach().cpu().numpy()) #input(C,W) output(original_seq_lenth,)
        layer2d_vlue_list.append(layer2d_vlue)
    for i in range(12,16):
        layer2d_vlue = cam.caculate_layer_cam_vlue_2d(fmap_block[i][0].clone().detach().cpu().numpy(),grad_block[15-i][0].clone().detach().cpu().numpy()) #input(C,H,W) output(lead,original_seq_lenth)
        layer2d_vlue_list.append(layer2d_vlue)
    if(((labels.tolist())[0]) == torch.tensor(1)):
        for i in range(12):
            branch_fmap_sum_HTN[i] += layer2d_vlue_list[i].sum()
        for i in range(12,24):
            branch_fmap_sum_HTN[i] += layer2d_vlue_list[12][i-12,:].sum()
        branch_fmap_sum_HTN[24] = fmap_block[13].to('cpu').clone().sum().detach().numpy()
        branch_fmap_sum_HTN[25] = fmap_block[14].to('cpu').clone().sum().detach().numpy()
        branch_fmap_sum_HTN[26] = fmap_block[15].to('cpu').clone().sum().detach().numpy()
    else:
        for i in range(12):
            branch_fmap_sum_NHTN[i] += layer2d_vlue_list[i].sum()
        for i in range(12,24):
            branch_fmap_sum_NHTN[i] += layer2d_vlue_list[12][i-12,:].sum()
        branch_fmap_sum_NHTN[24] = fmap_block[13].to('cpu').clone().sum().detach().numpy()
        branch_fmap_sum_NHTN[25] = fmap_block[14].to('cpu').clone().sum().detach().numpy()
        branch_fmap_sum_NHTN[26] = fmap_block[15].to('cpu').clone().sum().detach().numpy()
        
        
        
    # # print("labels: {}".format(labels))
    # # print("predict: {}".format(pred))
    # # loss = outputs[0,pred]      # 网络对应于pred的类别的输出即为loss
    # # # loss = (testmodel.last_out)[0,pred]
    # # loss.backward(retain_graph=True)  #retain_graph=True，目的为是为保留该过程中计算的梯度，后续G网络更新时使用
    # if(((labels.tolist())[0]) == torch.tensor(1)):
    #     for i in range(12):
    #         branch_fmap_sum_HTN[i]= branch_fmap_sum_HTN[i] + fmap_block[i].to('cpu').sum().detach().numpy()
    #         # print(fmap_block[i].size())
    #     for i in range(12,24):
    #         branch_fmap_sum_HTN[i]= branch_fmap_sum_HTN[i] + fmap_block[12][0,:,i-12].to('cpu').sum().detach().numpy()
    #     branch_fmap_sum_HTN[24] = fmap_block[13].to('cpu').sum().detach().numpy()
    #     branch_fmap_sum_HTN[25] = fmap_block[14].to('cpu').sum().detach().numpy()
    #     branch_fmap_sum_HTN[26] = fmap_block[15].to('cpu').sum().detach().numpy()
    # else:
    #     for i in range(12):
    #         branch_fmap_sum_NHTN[i]= branch_fmap_sum_NHTN[i] + fmap_block[i].to('cpu').sum().detach().numpy()
    #         # print(fmap_block[i].size())
    #     for i in range(12,24):
    #         branch_fmap_sum_NHTN[i]= branch_fmap_sum_NHTN[i] + fmap_block[12][0,:,i-12].to('cpu').sum().detach().numpy()
    #     branch_fmap_sum_NHTN[24] = fmap_block[13].to('cpu').sum().detach().numpy()
    #     branch_fmap_sum_NHTN[25] = fmap_block[14].to('cpu').sum().detach().numpy()
    #     branch_fmap_sum_NHTN[26] = fmap_block[15].to('cpu').sum().detach().numpy()

In [ ]:
import matplotlib.pyplot as plt

fig, (ax0, ax1) = plt.subplots(nrows = 2,ncols = 1,figsize=(12,10) ,sharex=True)  # type: ignore

leads = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6','I1', 'II1', 'III1', 'aVR1', 'aVL1', 'aVF1', 'V11', 'V21', 'V31', 'V41', 'V51', 'V61',]
counts = branch_fmap_sum_HTN[:24].tolist()
ax0.bar(leads, counts)
ax0.set_ylabel('fmap_sum_HTN')

counts = branch_fmap_sum_NHTN[:24].tolist()
ax1.bar(leads, counts)
ax1.set_ylabel('fmap_sum_NHTN')
plt.show()

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows = 2,ncols = 1,figsize=(12,10) ,sharex=True)  # type: ignore

leads = ['3', '5', '7']
counts = branch_fmap_sum_HTN[24:].tolist()
ax0.bar(leads, counts)
ax0.set_ylabel('fmap_sum_HTN')

counts = branch_fmap_sum_NHTN[24:].tolist()
ax1.bar(leads, counts)
ax1.set_ylabel('fmap_sum_NHTN')
plt.show()

layer cam

In [ ]:
NET = [Net.MLBFNet(num_class= 2,mark=True,res = True,se = True,Dropout_rate = 0.25) ] # type: ignore
testmodel = NET[0].to(DEVICE)
testmodel.load_state_dict(torch.load(Models_path))

In [ ]:
testmodel.eval()
# 定义获取梯度的函数
fmap_block = list()
grad_block = list()

# 获取反向的传播图
def backward_hook(module, grad_in, grad_out):
    grad_block.append(grad_out[0].detach())

# 定义获取特征图的函数
def farward_hook(module, input, output):
    fmap_block.append(output)

In [ ]:
testmodel.conv3.register_forward_hook(farward_hook)	#正向传播
testmodel.conv3.register_full_backward_hook(backward_hook)#反向传播

In [ ]:
def plot_top_attention_points(fig,axs,x,y,color_depend,cmap = "jet",y_name = "Voltage(mV)",title="",top_num = 100):
    axs.plot(x,y, color='black',linewidth=0.5)
    
    top_idx=color_depend.argsort()[::-1][0:top_num]
    axs.scatter(top_idx, y[top_idx],s=2,c='r')
    #fig.colorbar(line, ax=axs)
    axs.set_xlim(x.min(), x.max())
    axs.set_ylim(-3500, +3500)

    axs.set_aspect(0.2)#用于设置轴缩放的方面，即y-unit与x-unit的比率
    axs.xaxis.set_major_locator(plt.MultipleLocator(100))# type: ignore # 100*0.002s=0.2s = 5格
    axs.xaxis.set_minor_locator(plt.MultipleLocator(20)) # type: ignore # 20*0.002=0.004S = 1格
    axs.yaxis.set_major_locator(plt.MultipleLocator(500))# type: ignore # 0.1uv*500 = 0.5ms = 5格
    axs.yaxis.set_minor_locator(plt.MultipleLocator(100))# type: ignore # 0.1uv*100 =0.1ms = 1格 

    #axs.xaxis.set_major_formatter(plt.NullFormatter()) #x轴不显示刻度值/lable per 0.2s
    axs.xaxis.set_major_formatter(lambda x, pos: str(round(0.2*(x/100.0),2))) #x轴 lable per 0.2s
    axs.yaxis.set_major_formatter(lambda x, pos: str(x/1000.0)) # label per '0.5 mv'，turn uV to mv

    axs.grid(which='major', axis='x', linewidth=0.3, linestyle='-', color='b')
    axs.grid(which='minor', axis='x', linewidth=0.1, linestyle='-', color='b')
    axs.grid(which='major', axis='y', linewidth=0.3, linestyle='-', color='b')
    axs.grid(which='minor', axis='y', linewidth=0.1, linestyle='-', color='b')
    axs.set_ylabel(y_name)
    axs.set_title(title)
    axs.grid(True, which='both')

In [ ]:
for itme in tqdm(range(10)):
    fmap_block = list()
    grad_block = list()
    inputs,labels = test_dataset.__getitem__(itme)
    ECGFilename = (test_dataset.infos.iloc[itme]['ECGFilename'])
    labels = labels.unsqueeze(0) # 在首位添加1维作为batchsize
    inputs = inputs.unsqueeze(0) # 在首位添加1维作为batchsize
    testmodel.eval()
    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)  

    outputs = testmodel(inputs)
    _,pred = outputs.max(1)     # 求概率最大值对应的标签
    # print("labels: {}".format(labels))
    # print("predict: {}".format(pred))
    loss = outputs[0,pred]      # 网络对应于pred的类别的输出即为loss
    # loss = (testmodel.last_out)[0,pred]
    loss.backward(retain_graph=True)  #retain_graph=True，目的为是为保留该过程中计算的梯度，后续G网络更新时使用
    fmap = (fmap_block[0][0]).to('cpu').detach().numpy()
    gradmap = (grad_block[0][0]).to('cpu').detach().numpy()
    layer2d_vlue = cam.caculate_layer_cam_vlue_2d(fmap,gradmap)
    lead_index = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    x_index = np.arange(0,EcgLength_num)
    fig, axs = plt.subplots(nrows=6, ncols=2, sharex=True,sharey=True,figsize=(50,60), constrained_layout=True)
    ecg_data = (inputs[0]).to('cpu')
    for i,ax in enumerate(axs.flat):  # type: ignore
        attention_value_each_timestep = layer2d_vlue[i]
        #plot_y = x[1,i,:]*(4.88)
        plot_y = np.array(ecg_data[i]*3500.)
        ECGplot.plot_multicolored_line(fig,ax,x = x_index,y= plot_y,color_depend=attention_value_each_timestep,cmap="jet",y_name = str(lead_index[i])+" Voltage(mV)",title = lead_index[i])
        plot_top_attention_points(fig,ax,x = x_index,y= plot_y,color_depend=attention_value_each_timestep,cmap="jet",y_name = str(lead_index[i])+" Voltage(mV)",title = lead_index[i],top_num=500)
    plt.savefig('/workspace/data/Interpretable_HTN/model/20230131_114246/20230131_114246/'+'/'+str(ECGFilename)+'_'+str(labels[0].tolist())+'_'+str(pred[0].tolist())+'_''.jpg', bbox_inches='tight',dpi = 100)

# single sampel

In [ ]:
NET = [Net.MLBFNet(num_class=2,mark=True,res = True,se = True,Dropout_rate = 0.3) ] # type: ignore
testmodel = NET[0].to(DEVICE)
testmodel.load_state_dict(torch.load(Models_path))

In [ ]:
# 定义获取梯度的函数
fmap_block = list()
grad_block = list()

# 获取反向的传播图
def backward_hook(module, grad_in, grad_out):
    # print('backward_hook ',len(grad_out),grad_out[0].shape)
    grad_block.append(grad_out[0].clone().cpu().detach())

# 定义获取特征图的函数
def farward_hook(module, input, output):
    # print('farward_hook ',output.shape)
    fmap_block.append(output.clone().cpu().detach())

In [ ]:
index = 2

testmodel = NET[0].to(DEVICE)
testmodel.load_state_dict(torch.load(Models_path))

# testmodel.conv3.register_forward_hook(farward_hook)	#正向传播
# testmodel.conv3.register_full_backward_hook(backward_hook)#反向传播
testmodel.conv3.register_forward_hook(farward_hook)	#正向传播
testmodel.conv3.register_full_backward_hook(backward_hook)#反向传播


inputs,labels = test_dataset.__getitem__(index)
info =test_dataset.infos.iloc[index]
# print(inputs,info)
testmodel.eval()
fmap_block = list()
grad_block = list()

labels = labels.unsqueeze(0) # 在首位添加1维作为batchsize
inputs = inputs.unsqueeze(0) # 在首位添加1维作为batchsize

inputs = inputs.to(DEVICE)
labels = labels.to(DEVICE)  

outputs = testmodel(inputs)
# print('outputs',outputs)
_,pred = outputs.max(1)     # 求概率最大值对应的标签
possibility = nn.functional.softmax(outputs,dim=-1)
print("labels: {}".format(labels))
print("predict: {}".format(pred))
print("possibility: {}".format(possibility))
loss = outputs[0,pred]      # 网络对应于pred的类别的输出即为loss
loss.backward(retain_graph=True)  #retain_graph=True，目的为是为保留该过程中计算的梯度，后续G网络更新时使用
fmap = (fmap_block[0][0]).to('cpu').detach().numpy()
gradmap = (grad_block[0][0]).to('cpu').detach().numpy()

In [ ]:
layer2d_vlue = cam.caculate_layer_cam_vlue_2d(fmap,gradmap)

In [ ]:
layer2d_vlue.shape

In [ ]:
layer2d_vlue = cam.caculate_layer_cam_vlue_2d(fmap,gradmap)
lead_index = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
x_index = np.arange(0,12)
fig, axs = plt.subplots(nrows=4, ncols=3, sharex=True,sharey=True,figsize=(45,25), constrained_layout=True,dpi =300)
ecg_data,_ = test_dataset.__getitem__(index)

cmap = 'OrRd' 
colors = ECGplot.color_map(layer2d_vlue, cmap)#相当于归一化

for i,ax in enumerate(axs.flat):  # type: ignore
    plot_y = np.array(ecg_data[i]*5000.)
    x = np.arange(0,5000)
    ECGplot.plot_ECG_line(fig,ax,x = x,y= plot_y,y_name = str(lead_index[i])+" Voltage(mV)",title = lead_index[i])
    for j in x:
        ax.axvline(x=j,alpha=0.1,color=colors[i,j]) #第i导联 第j个时间点的importance value
    

In [ ]:
NET = [Net.MLBFNet_GUR(mark = True,res = True,se = True,Dropout_rate = 0.3) ] # type: ignore
testmodel = NET[0].to(DEVICE)
Models_path = '/workspace/data/Interpretable_HTN/model/20230201_143012/20230201_143012/parameter_EarlyStoping_2.pt'
testmodel.load_state_dict(torch.load(Models_path))
save_root = '/workspace/data/Interpretable_HTN/model/20230201_143012/20230201_143012/parameter_EarlyStoping_2/'

# 定义获取梯度的函数
fmap_block = list()
grad_block = list()

# 获取反向的传播图
def backward_hook(module, grad_in, grad_out):
    print('backward_hook ',len(grad_out),grad_out[0].shape)
    grad_block.append(grad_out[0].clone().cpu().detach())

# 定义获取特征图的函数
def farward_hook(module, input, output):
    print('farward_hook ',output.shape)
    fmap_block.append(output.clone().cpu().detach())
    
    
for index in range(test_dataset.__len__()):
    testmodel = NET[0].to(DEVICE)
    
    testmodel.load_state_dict(torch.load(Models_path))

    # testmodel.conv3.register_forward_hook(farward_hook)	#正向传播
    # testmodel.conv3.register_full_backward_hook(backward_hook)#反向传播
    testmodel.conv3.register_forward_hook(farward_hook)	#正向传播
    testmodel.conv3.register_full_backward_hook(backward_hook)#反向传播


    inputs,labels = test_dataset.__getitem__(index)
    info =test_dataset.infos.iloc[index]
    # print(inputs,info)
    testmodel.eval()
    fmap_block = list()
    grad_block = list()

    labels = labels.unsqueeze(0) # 在首位添加1维作为batchsize
    inputs = inputs.unsqueeze(0) # 在首位添加1维作为batchsize

    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)  

    outputs = testmodel(inputs)
    # print('outputs',outputs)
    _,pred = outputs.max(1)     # 求概率最大值对应的标签
    possibility = nn.functional.softmax(outputs,dim=-1)
    print("labels: {}".format(labels))
    print("predict: {}".format(pred))
    print("possibility: {}".format(possibility))
    loss = outputs[0,pred]      # 网络对应于pred的类别的输出即为loss
    loss.backward(retain_graph=True)  #retain_graph=True，目的为是为保留该过程中计算的梯度，后续G网络更新时使用
    fmap = (fmap_block[0][0]).to('cpu').detach().numpy()
    gradmap = (grad_block[0][0]).to('cpu').detach().numpy()

    layer2d_vlue = cam.caculate_layer_cam_vlue_2d(fmap,gradmap)
    lead_index = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    x_index = np.arange(0,12)
    fig, axs = plt.subplots(nrows=4, ncols=3, sharex=True,sharey=True,figsize=(45,25), constrained_layout=True,dpi =100)
    ecg_data,_ = test_dataset.__getitem__(index)

    cmap = 'OrRd' 
    colors = ECGplot.color_map(layer2d_vlue, cmap)#相当于归一化

    for i,ax in enumerate(axs.flat):  # type: ignore
        plot_y = np.array(ecg_data[i]*5000.)#缩放
        t = np.arange(0,5000)
        ECGplot.plot_ECG_line(fig,ax,x = t,y= plot_y,y_name = str(lead_index[i])+" Voltage(mV)",title = lead_index[i])
        for j in t:
            ax.axvline(x=j,alpha=0.1,color=colors[i,j]) #第i导联 第j个时间点的importance value
    plt.savefig(save_root+'/'+info['ECGFilename']+'_'+str(labels[0].tolist())+'_'+str(pred[0].tolist())+'_'+'conv3'+'.png', bbox_inches='tight',dpi = 200)   
    plt.close()

In [ ]:
NET = [Net.MLBFNet(num_class = 2,mark = True,res = True,se = True,Dropout_rate = 0.3) ] # type: ignore
testmodel = NET[0].to(DEVICE)
Models_path = '/workspace/data/Interpretable_HTN/model/20230131_114246/20230131_114246/parameter_EarlyStoping_0.pt'
testmodel.load_state_dict(torch.load(Models_path))
save_root = '/workspace/data/Interpretable_HTN/model/20230131_114246/20230131_114246/parameter_EarlyStoping_0/'

# 定义获取梯度的函数
fmap_block = list()
grad_block = list()

# 获取反向的传播图
def backward_hook(module, grad_in, grad_out):
    print('backward_hook ',len(grad_out),grad_out[0].shape)
    grad_block.append(grad_out[0].clone().cpu().detach())

# 定义获取特征图的函数
def farward_hook(module, input, output):
    print('farward_hook ',output.shape)
    fmap_block.append(output.clone().cpu().detach())
    
    
for index in range(20):
    NET = [Net.MLBFNet(num_class = 2,mark = True,res = True,se = True,Dropout_rate = 0.3) ] # type: ignore
    testmodel = NET[0].to(DEVICE)
    testmodel.load_state_dict(torch.load(Models_path))

    # testmodel.conv3.register_forward_hook(farward_hook)	#正向传播
    # testmodel.conv3.register_full_backward_hook(backward_hook)#反向传播
    testmodel.conv3.register_forward_hook(farward_hook)	#正向传播
    testmodel.conv3.register_full_backward_hook(backward_hook)#反向传播


    inputs,labels = test_dataset.__getitem__(index)
    info =test_dataset.infos.iloc[index]
    # print(inputs,info)
    testmodel.eval()
    fmap_block = list()
    grad_block = list()

    labels = labels.unsqueeze(0) # 在首位添加1维作为batchsize
    inputs = inputs.unsqueeze(0) # 在首位添加1维作为batchsize

    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)  

    outputs = testmodel(inputs)
    # print('outputs',outputs)
    _,pred = outputs.max(1)     # 求概率最大值对应的标签
    possibility = nn.functional.softmax(outputs,dim=-1)
    print("labels: {}".format(labels))
    print("predict: {}".format(pred))
    print("possibility: {}".format(possibility))
    loss = outputs[0,pred]      # 网络对应于pred的类别的输出即为loss
    loss.backward(retain_graph=True)  #retain_graph=True，目的为是为保留该过程中计算的梯度，后续G网络更新时使用
    fmap = (fmap_block[0][0]).to('cpu').detach().numpy()
    gradmap = (grad_block[0][0]).to('cpu').detach().numpy()

    layer2d_vlue = cam.caculate_layer_cam_vlue_2d(fmap,gradmap)
    lead_index = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
    x_index = np.arange(0,12)
    fig, axs = plt.subplots(nrows=4, ncols=3, sharex=True,sharey=True,figsize=(45,25), constrained_layout=True,dpi =100)
    ecg_data,_ = test_dataset.__getitem__(index)

    cmap = 'OrRd' 
    colors = ECGplot.color_map(layer2d_vlue, cmap)#相当于归一化

    for i,ax in enumerate(axs.flat):  # type: ignore
        plot_y = np.array(ecg_data[i]*5000.)#缩放
        t = np.arange(0,5000)
        ECGplot.plot_ECG_line(fig,ax,x = t,y= plot_y,y_name = str(lead_index[i])+" Voltage(mV)",title = lead_index[i])
        for j in t:
            ax.axvline(x=j,alpha=0.1,color=colors[i,j]) #第i导联 第j个时间点的importance value
    plt.savefig(save_root+'/'+info['ECGFilename']+'_'+str(labels[0].tolist())+'_'+str(pred[0].tolist())+'_'+'conv3'+'.png', bbox_inches='tight',dpi = 200)  
    plt.close()  